In [8]:
# Importing the required Libraries and packages
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from nltk import word_tokenize
import string
import re
import nltk

In [9]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [11]:
os.getcwd()

'/Users/sumitbajare/Documents/Sussex/Project/model'

In [12]:
os.chdir('/Users/sumitbajare/Documents/Sussex/Project')

In [13]:
# Loading the Kiswa corpus Training dataset for processing
data = pd.read_csv("data/kiswa_train.csv")
# preview the top 5 rows of the dataset
data.head()

,date,id,news_heading_eng,news_heading_swa,news_type
0,2021-08-12,bbc567,william ruto : a chicken seller who rose throu...,william ruto : mchuuzi wa kuku aliyepanda vyeo...,Local News
1,2021-05-23,tl1446,the number of young people in agriculture and ...,idadi ya vijana katika kilimo na ufugaji wa wa...,Local News
2,2021-10-20,tl1056,the court was told how a woman stole from a bu...,mahakama ilielezwa jinsi mwanamke alivyoiba ku...,Local News
3,2021-07-20,tl539,man accused of rape .,mtuhumiwa wa ubakaji .,Local News
4,2021-03-07,tl1670,uhuru orders kdf land to be returned to the sa...,uhuru aagiza ardhi ya kdf irejeshwe kwa jamii ...,Local News


In [14]:
data.head()

,date,id,news_heading_eng,news_heading_swa,news_type
0,2021-08-12,bbc567,william ruto : a chicken seller who rose throu...,william ruto : mchuuzi wa kuku aliyepanda vyeo...,Local News
1,2021-05-23,tl1446,the number of young people in agriculture and ...,idadi ya vijana katika kilimo na ufugaji wa wa...,Local News
2,2021-10-20,tl1056,the court was told how a woman stole from a bu...,mahakama ilielezwa jinsi mwanamke alivyoiba ku...,Local News
3,2021-07-20,tl539,man accused of rape .,mtuhumiwa wa ubakaji .,Local News
4,2021-03-07,tl1670,uhuru orders kdf land to be returned to the sa...,uhuru aagiza ardhi ya kdf irejeshwe kwa jamii ...,Local News


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2233 entries, 0 to 2232
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date              2233 non-null   object
 1   id                2233 non-null   object
 2   news_heading_eng  2233 non-null   object
 3   news_heading_swa  2232 non-null   object
 4   news_type         2233 non-null   object
dtypes: object(5)
memory usage: 87.4+ KB


In [7]:
data.shape

(2233, 5)

In [8]:
def clean_text(sentence):
    '''
        This function cleans the column
    '''
    sentence = sentence.lower()                #convert text to lower-case
    sentence = re.sub('[‘’“”…,]', '', sentence) # remove punctuation
    sentence = re.sub('[()]', '', sentence)  #remove parentheses
    sentence = word_tokenize(sentence)      # remove repeated characters (tanzaniaaaaaaaa to tanzania)
    
    return ' '.join(sentence)

In [9]:
# converting the float into string. 
data = data.astype(str)

In [10]:
# calling the clean_text function on news_heading_swa
data['news_heading_swa'] = data['news_heading_swa'].apply(clean_text) 
data['news_heading_eng'] = data['news_heading_eng'].apply(clean_text) 
data.head()

,date,id,news_heading_eng,news_heading_swa,news_type
0,2021-08-12,bbc567,william ruto : a chicken seller who rose throu...,william ruto : mchuuzi wa kuku aliyepanda vyeo...,Local News
1,2021-05-23,tl1446,the number of young people in agriculture and ...,idadi ya vijana katika kilimo na ufugaji wa wa...,Local News
2,2021-10-20,tl1056,the court was told how a woman stole from a bu...,mahakama ilielezwa jinsi mwanamke alivyoiba ku...,Local News
3,2021-07-20,tl539,man accused of rape .,mtuhumiwa wa ubakaji .,Local News
4,2021-03-07,tl1670,uhuru orders kdf land to be returned to the sa...,uhuru aagiza ardhi ya kdf irejeshwe kwa jamii ...,Local News


In [11]:
# Spliting the data into training and validation
df = data[['news_type', 'news_heading_swa']] 
SEED = 2020
df_train = df.sample(frac=0.89, random_state=SEED)
df_test = df.drop(df_train.index)
len(df_train), len(df_test)

(1987, 246)

In [14]:
import ktrain
from ktrain import text

# selecting transformer to use
MODEL_NAME = 'bert-base-multilingual-uncased'

# Common hyper-parameters
MAXLEN  = 136 # Only consider the first 136 words of each news content
batch_size = 64 #  the number of training examples utilised in one iteration.
learning_rate = 4e-5 # the amount that the weights are updated during training
epochs = 10 # the number of passes of the entire training dataset the algorithm has completed
t = text.Transformer(MODEL_NAME, maxlen = MAXLEN)
trn = t.preprocess_train(df_train.news_heading_swa.values, df_train.news_type.values)
val = t.preprocess_test(df_test.news_heading_swa.values, df_test.news_type.values)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=batch_size)
learner.fit(learning_rate, epochs)

preprocessing train...
language: sw
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 19


Is Multi-Label? False
preprocessing test...
language: sw
test sequence lengths:
	mean : 10
	95percentile : 15
	99percentile : 18


Epoch 1/10
32/32 [==============================] - 937s 29s/step - loss: 0.9043 - accuracy: 0.7967 - val_loss: 0.8581 - val_accuracy: 0.7764
Epoch 2/10
32/32 [==============================] - 895s 28s/step - loss: 0.6712 - accuracy: 0.8249 - val_loss: 0.7783 - val_accuracy: 0.8049
Epoch 3/10
32/32 [==============================] - 865s 27s/step - loss: 0.5981 - accuracy: 0.8410 - val_loss: 0.6778 - val_accuracy: 0.8130
Epoch 4/10
32/32 [==============================] - 858s 27s/step - loss: 0.5240 - accuracy: 0.8500 - val_loss: 0.6677 - val_accuracy: 0.8293
Epoch 5/10
32/32 [==============================] - 862s 27s/step - loss: 0.4181 - accuracy: 0.8908 - val_loss: 0.6665 - val_accuracy: 0.8252
Epoch 6/10
32/32 [==============================] - 833s 26s/step - loss: 0.3593 - accuracy: 0.9034 - val_loss: 0.6717 - val_accuracy: 0.8293
Epoch 7/10
32/32 [==============================] - 846s 26s/step - loss: 0.3100 - accuracy: 0.9134 - val_loss: 0.7214 - val_accuracy: 0.8374
Epoch 

In [15]:
import ktrain
from ktrain import text

# selecting transformer to use
MODEL_NAME = 'bert-base-multilingual-uncased'

# Common hyper-parameters
MAXLEN  = 136 # Only consider the first 136 words of each news content
batch_size = 128 #  the number of training examples utilised in one iteration.
learning_rate = 4e-5 # the amount that the weights are updated during training
epochs = 10 # the number of passes of the entire training dataset the algorithm has completed
t = text.Transformer(MODEL_NAME, maxlen = MAXLEN)
trn = t.preprocess_train(df_train.news_heading_swa.values, df_train.news_type.values)
val = t.preprocess_test(df_test.news_heading_swa.values, df_test.news_type.values)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=batch_size)
learner.fit(learning_rate, epochs)

preprocessing train...
language: sw
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 19


Is Multi-Label? False
preprocessing test...
language: sw
test sequence lengths:
	mean : 10
	95percentile : 15
	99percentile : 18


Epoch 1/10
16/16 [==============================] - 1238s 76s/step - loss: 1.1009 - accuracy: 0.6759 - val_loss: 0.9071 - val_accuracy: 0.7764
Epoch 2/10
16/16 [==============================] - 1183s 74s/step - loss: 0.7978 - accuracy: 0.8032 - val_loss: 0.8854 - val_accuracy: 0.7764
Epoch 3/10
16/16 [==============================] - 1036s 64s/step - loss: 0.7728 - accuracy: 0.8032 - val_loss: 0.8317 - val_accuracy: 0.7927
Epoch 4/10
16/16 [==============================] - 1003s 63s/step - loss: 0.6774 - accuracy: 0.8223 - val_loss: 0.7356 - val_accuracy: 0.7927
Epoch 5/10
16/16 [==============================] - 989s 62s/step - loss: 0.5736 - accuracy: 0.8505 - val_loss: 0.6992 - val_accuracy: 0.8089
Epoch 6/10
16/16 [==============================] - 969s 61s/step - loss: 0.5055 - accuracy: 0.8656 - val_loss: 0.6987 - val_accuracy: 0.8293
Epoch 7/10
16/16 [==============================] - 1021s 64s/step - loss: 0.4528 - accuracy: 0.8792 - val_loss: 0.7012 - val_accuracy: 0.7886
E

In [11]:
import ktrain
from ktrain import text

# selecting transformer to use
MODEL_NAME = 'bert-base-multilingual-uncased'

# Common hyper-parameters
MAXLEN  = 136 # Only consider the first 136 words of each news content
batch_size = 128 #  the number of training examples utilised in one iteration.
learning_rate = 5e-5 # the amount that the weights are updated during training
epochs = 10 # the number of passes of the entire training dataset the algorithm has completed
t = text.Transformer(MODEL_NAME, maxlen = MAXLEN)
trn = t.preprocess_train(df_train.news_heading_swa.values, df_train.news_type.values)
val = t.preprocess_test(df_test.news_heading_swa.values, df_test.news_type.values)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=batch_size)
learner.fit(learning_rate, epochs)

preprocessing train...
language: sw
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 19


Is Multi-Label? False
preprocessing test...
language: sw
test sequence lengths:
	mean : 10
	95percentile : 15
	99percentile : 18


Epoch 1/10
16/16 [==============================] - 1071s 66s/step - loss: 0.9708 - accuracy: 0.7952 - val_loss: 0.9062 - val_accuracy: 0.7764
Epoch 2/10
16/16 [==============================] - 1050s 65s/step - loss: 0.8026 - accuracy: 0.8032 - val_loss: 0.8995 - val_accuracy: 0.7764
Epoch 3/10
16/16 [==============================] - 1062s 67s/step - loss: 0.7480 - accuracy: 0.8027 - val_loss: 0.8340 - val_accuracy: 0.7764
Epoch 4/10
16/16 [==============================] - 1011s 63s/step - loss: 0.6202 - accuracy: 0.8319 - val_loss: 0.7260 - val_accuracy: 0.8171
Epoch 5/10
16/16 [==============================] - 1063s 66s/step - loss: 0.5098 - accuracy: 0.8646 - val_loss: 0.7195 - val_accuracy: 0.8171
Epoch 6/10
16/16 [==============================] - 1073s 67s/step - loss: 0.4556 - accuracy: 0.8671 - val_loss: 0.7123 - val_accuracy: 0.8252
Epoch 7/10
16/16 [==============================] - 1189s 75s/step - loss: 0.3478 - accuracy: 0.8983 - val_loss: 0.6972 - val_accuracy: 0.8211

In [12]:
import ktrain
from ktrain import text

# selecting transformer to use
MODEL_NAME = 'bert-base-multilingual-uncased'

# Common hyper-parameters
MAXLEN  = 136 # Only consider the first 136 words of each news content
batch_size = 64 #  the number of training examples utilised in one iteration.
learning_rate = 5e-5 # the amount that the weights are updated during training
epochs = 10 # the number of passes of the entire training dataset the algorithm has completed
t = text.Transformer(MODEL_NAME, maxlen = MAXLEN)
trn = t.preprocess_train(df_train.news_heading_swa.values, df_train.news_type.values)
val = t.preprocess_test(df_test.news_heading_swa.values, df_test.news_type.values)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=batch_size)
learner.fit(learning_rate, epochs)

preprocessing train...
language: sw
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 19


Is Multi-Label? False
preprocessing test...
language: sw
test sequence lengths:
	mean : 10
	95percentile : 15
	99percentile : 18


Epoch 1/10
32/32 [==============================] - 988s 30s/step - loss: 1.0004 - accuracy: 0.7650 - val_loss: 0.9256 - val_accuracy: 0.7764
Epoch 2/10
32/32 [==============================] - 954s 30s/step - loss: 0.7967 - accuracy: 0.8032 - val_loss: 0.9613 - val_accuracy: 0.7764
Epoch 3/10
32/32 [==============================] - 905s 28s/step - loss: 0.8053 - accuracy: 0.8032 - val_loss: 0.8502 - val_accuracy: 0.7764
Epoch 4/10
32/32 [==============================] - 895s 28s/step - loss: 0.7170 - accuracy: 0.8027 - val_loss: 0.7549 - val_accuracy: 0.7886
Epoch 5/10
32/32 [==============================] - 920s 29s/step - loss: 0.5909 - accuracy: 0.8269 - val_loss: 0.7182 - val_accuracy: 0.7886
Epoch 6/10
32/32 [==============================] - 980s 31s/step - loss: 0.5063 - accuracy: 0.8520 - val_loss: 0.8008 - val_accuracy: 0.8130
Epoch 7/10
32/32 [==============================] - 988s 31s/step - loss: 0.4159 - accuracy: 0.8878 - val_loss: 0.7165 - val_accuracy: 0.8049
Epoch 

In [13]:
import ktrain
from ktrain import text

# selecting transformer to use
MODEL_NAME = 'bert-base-multilingual-uncased'

# Common hyper-parameters
MAXLEN  = 136 # Only consider the first 136 words of each news content
batch_size = 32 #  the number of training examples utilised in one iteration.
learning_rate = 5e-5 # the amount that the weights are updated during training
epochs = 7 # the number of passes of the entire training dataset the algorithm has completed
t = text.Transformer(MODEL_NAME, maxlen = MAXLEN)
trn = t.preprocess_train(df_train.news_heading_swa.values, df_train.news_type.values)
val = t.preprocess_test(df_test.news_heading_swa.values, df_test.news_type.values)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=batch_size)
learner.fit(learning_rate, epochs)

preprocessing train...
language: sw
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 19


Is Multi-Label? False
preprocessing test...
language: sw
test sequence lengths:
	mean : 10
	95percentile : 15
	99percentile : 18


Epoch 1/7
63/63 [==============================] - 804s 13s/step - loss: 0.8403 - accuracy: 0.7997 - val_loss: 0.8730 - val_accuracy: 0.7764
Epoch 2/7
63/63 [==============================] - 796s 13s/step - loss: 0.7037 - accuracy: 0.8052 - val_loss: 0.7761 - val_accuracy: 0.7764
Epoch 3/7
63/63 [==============================] - 823s 13s/step - loss: 0.6490 - accuracy: 0.8078 - val_loss: 0.7367 - val_accuracy: 0.7886
Epoch 4/7
63/63 [==============================] - 791s 13s/step - loss: 0.5590 - accuracy: 0.8420 - val_loss: 0.7922 - val_accuracy: 0.7846
Epoch 5/7
63/63 [==============================] - 759s 12s/step - loss: 0.4782 - accuracy: 0.8702 - val_loss: 0.7222 - val_accuracy: 0.8008
Epoch 6/7
63/63 [==============================] - 798s 13s/step - loss: 0.3881 - accuracy: 0.8863 - val_loss: 0.7501 - val_accuracy: 0.7561
Epoch 7/7
63/63 [==============================] - 810s 13s/step - loss: 0.3351 - accuracy: 0.8988 - val_loss: 0.8244 - val_accuracy: 0.8211


In [14]:
import ktrain
from ktrain import text

# selecting transformer to use
MODEL_NAME = 'bert-base-multilingual-uncased'

# Common hyper-parameters
MAXLEN  = 136 # Only consider the first 136 words of each news content
batch_size = 32 #  the number of training examples utilised in one iteration.
learning_rate = 4e-5 # the amount that the weights are updated during training
epochs = 7 # the number of passes of the entire training dataset the algorithm has completed
t = text.Transformer(MODEL_NAME, maxlen = MAXLEN)
trn = t.preprocess_train(df_train.news_heading_swa.values, df_train.news_type.values)
val = t.preprocess_test(df_test.news_heading_swa.values, df_test.news_type.values)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=batch_size)
learner.fit(learning_rate, epochs)

preprocessing train...
language: sw
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 19


Is Multi-Label? False
preprocessing test...
language: sw
test sequence lengths:
	mean : 10
	95percentile : 15
	99percentile : 18


Epoch 1/7
63/63 [==============================] - 747s 12s/step - loss: 0.8355 - accuracy: 0.7921 - val_loss: 0.8009 - val_accuracy: 0.7886
Epoch 2/7
63/63 [==============================] - 698s 11s/step - loss: 0.6228 - accuracy: 0.8354 - val_loss: 0.6635 - val_accuracy: 0.8333
Epoch 3/7
63/63 [==============================] - 697s 11s/step - loss: 0.5396 - accuracy: 0.8591 - val_loss: 0.6546 - val_accuracy: 0.8293
Epoch 4/7
63/63 [==============================] - 699s 11s/step - loss: 0.4814 - accuracy: 0.8702 - val_loss: 0.7367 - val_accuracy: 0.8293
Epoch 5/7
63/63 [==============================] - 749s 12s/step - loss: 0.3962 - accuracy: 0.8908 - val_loss: 0.7061 - val_accuracy: 0.8211
Epoch 6/7
63/63 [==============================] - 1008s 16s/step - loss: 0.3139 - accuracy: 0.9134 - val_loss: 0.6311 - val_accuracy: 0.8415
Epoch 7/7
63/63 [==============================] - 892s 14s/step - loss: 0.2383 - accuracy: 0.9366 - val_loss: 0.8356 - val_accuracy: 0.8130


In [12]:
import ktrain
from ktrain import text

# selecting transformer to use
MODEL_NAME = 'bert-base-multilingual-uncased'

# Common hyper-parameters
MAXLEN  = 136 # Only consider the first 136 words of each news content
batch_size = 32 #  the number of training examples utilised in one iteration.
learning_rate = 4e-5 # the amount that the weights are updated during training
epochs = 10 # the number of passes of the entire training dataset the algorithm has completed
t = text.Transformer(MODEL_NAME, maxlen = MAXLEN)
trn = t.preprocess_train(df_train.news_heading_swa.values, df_train.news_type.values)
val = t.preprocess_test(df_test.news_heading_swa.values, df_test.news_type.values)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=batch_size)
learner.fit(learning_rate, epochs)

preprocessing train...
language: sw
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 19


Is Multi-Label? False
preprocessing test...
language: sw
test sequence lengths:
	mean : 10
	95percentile : 15
	99percentile : 18


Epoch 1/10
63/63 [==============================] - 728s 11s/step - loss: 0.8320 - accuracy: 0.7977 - val_loss: 0.8039 - val_accuracy: 0.7967
Epoch 2/10
63/63 [==============================] - 675s 11s/step - loss: 0.6237 - accuracy: 0.8344 - val_loss: 0.7235 - val_accuracy: 0.7967
Epoch 3/10
63/63 [==============================] - 726s 12s/step - loss: 0.5147 - accuracy: 0.8571 - val_loss: 0.6910 - val_accuracy: 0.8089
Epoch 4/10
63/63 [==============================] - 712s 11s/step - loss: 0.4289 - accuracy: 0.8802 - val_loss: 0.6400 - val_accuracy: 0.8089
Epoch 5/10
63/63 [==============================] - 700s 11s/step - loss: 0.3604 - accuracy: 0.8918 - val_loss: 0.7147 - val_accuracy: 0.8171
Epoch 6/10
63/63 [==============================] - 676s 11s/step - loss: 0.2986 - accuracy: 0.9170 - val_loss: 0.7664 - val_accuracy: 0.8089
Epoch 7/10
63/63 [==============================] - 697s 11s/step - loss: 0.2635 - accuracy: 0.9235 - val_loss: 0.7355 - val_accuracy: 0.8211
Epoch 

In [14]:
import ktrain
from ktrain import text

# selecting transformer to use
MODEL_NAME = 'bert-base-multilingual-uncased'

# Common hyper-parameters
MAXLEN  = 136 # Only consider the first 136 words of each news content
batch_size = 64 #  the number of training examples utilised in one iteration.
learning_rate = 4e-5 # the amount that the weights are updated during training
epochs = 3 # the number of passes of the entire training dataset the algorithm has completed
t = text.Transformer(MODEL_NAME, maxlen = MAXLEN)
trn = t.preprocess_train(df_train.news_heading_swa.values, df_train.news_type.values)
val = t.preprocess_test(df_test.news_heading_swa.values, df_test.news_type.values)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=batch_size)
learner.fit(learning_rate, epochs)

preprocessing train...
language: sw
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 19


Is Multi-Label? False
preprocessing test...
language: sw
test sequence lengths:
	mean : 10
	95percentile : 15
	99percentile : 18


Epoch 1/3
32/32 [==============================] - 915s 28s/step - loss: 0.9179 - accuracy: 0.7791 - val_loss: 0.8826 - val_accuracy: 0.7764
Epoch 2/3
32/32 [==============================] - 960s 30s/step - loss: 0.7337 - accuracy: 0.8093 - val_loss: 0.8163 - val_accuracy: 0.7602
Epoch 3/3
32/32 [==============================] - 948s 30s/step - loss: 0.6398 - accuracy: 0.8254 - val_loss: 0.7482 - val_accuracy: 0.7846


In [15]:
import ktrain
from ktrain import text

# selecting transformer to use
MODEL_NAME = 'bert-base-multilingual-uncased'

# Common hyper-parameters
MAXLEN  = 136 # Only consider the first 136 words of each news content
batch_size = 64 #  the number of training examples utilised in one iteration.
learning_rate = 5e-5 # the amount that the weights are updated during training
epochs = 3 # the number of passes of the entire training dataset the algorithm has completed
t = text.Transformer(MODEL_NAME, maxlen = MAXLEN)
trn = t.preprocess_train(df_train.news_heading_swa.values, df_train.news_type.values)
val = t.preprocess_test(df_test.news_heading_swa.values, df_test.news_type.values)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=batch_size)
learner.fit(learning_rate, epochs)

preprocessing train...
language: sw
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 19


Is Multi-Label? False
preprocessing test...
language: sw
test sequence lengths:
	mean : 10
	95percentile : 15
	99percentile : 18


Epoch 1/3
32/32 [==============================] - 933s 29s/step - loss: 0.8706 - accuracy: 0.7977 - val_loss: 0.8731 - val_accuracy: 0.7764
Epoch 2/3
32/32 [==============================] - 946s 30s/step - loss: 0.6749 - accuracy: 0.8153 - val_loss: 0.7389 - val_accuracy: 0.7967
Epoch 3/3
32/32 [==============================] - 977s 30s/step - loss: 0.5566 - accuracy: 0.8485 - val_loss: 0.6893 - val_accuracy: 0.8211


# Selecting the best model 

In [16]:
import ktrain
from ktrain import text

# selecting transformer to use
MODEL_NAME = 'bert-base-multilingual-uncased'

# Common hyper-parameters
MAXLEN  = 136 # Only consider the first 136 words of each news content
batch_size = 64 #  the number of training examples utilised in one iteration.
learning_rate = 4e-5 # the amount that the weights are updated during training
epochs = 10 # the number of passes of the entire training dataset the algorithm has completed
t = text.Transformer(MODEL_NAME, maxlen = MAXLEN)
trn = t.preprocess_train(df_train.news_heading_swa.values, df_train.news_type.values)
val = t.preprocess_test(df_test.news_heading_swa.values, df_test.news_type.values)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=batch_size)
learner.fit(learning_rate, epochs)

preprocessing train...
language: sw
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 19


Is Multi-Label? False
preprocessing test...
language: sw
test sequence lengths:
	mean : 10
	95percentile : 15
	99percentile : 18


Epoch 1/10
32/32 [==============================] - 948s 29s/step - loss: 0.8961 - accuracy: 0.8022 - val_loss: 0.8631 - val_accuracy: 0.7764
Epoch 2/10
32/32 [==============================] - 841s 26s/step - loss: 0.6923 - accuracy: 0.8183 - val_loss: 0.7956 - val_accuracy: 0.8008
Epoch 3/10
32/32 [==============================] - 827s 26s/step - loss: 0.5695 - accuracy: 0.8510 - val_loss: 0.7107 - val_accuracy: 0.8333
Epoch 4/10
32/32 [==============================] - 828s 26s/step - loss: 0.4504 - accuracy: 0.8787 - val_loss: 0.7245 - val_accuracy: 0.8171
Epoch 5/10
32/32 [==============================] - 830s 26s/step - loss: 0.3338 - accuracy: 0.9044 - val_loss: 0.7251 - val_accuracy: 0.8049
Epoch 6/10
32/32 [==============================] - 830s 26s/step - loss: 0.2655 - accuracy: 0.9225 - val_loss: 0.7771 - val_accuracy: 0.7805
Epoch 7/10
32/32 [==============================] - 904s 28s/step - loss: 0.1936 - accuracy: 0.9441 - val_loss: 0.8173 - val_accuracy: 0.8293
Epoch 

# Testing the model

In [17]:
kiswa_classifier = ktrain.get_predictor(learner.model,t)

In [18]:
# Fecthing the current directory
os.getcwd()

'/Users/sumitbajare/Documents/Sussex/Project'

In [19]:
# Changing the directory
os.chdir('/Users/sumitbajare/Documents/Sussex/Project/data/')

In [7]:
def test(file):
    '''
        This function will read the contents of a part of corpus and 
        print the new type of the content.
        
        Parameter: file- the file where the contents are stored
        Return: contents, news_type
    '''
    
    # extracting a specific news from the kiswa corpus 
    test_file=open(file,'r')
    # reading the whole file in a string
    content=test_file.read()
    # closing the file
    test_file.close()
    # prdicting the type of the news
    news_type=kiswa_classifier.predict(clean_text(content))
    return content,news_type

In [16]:
# Loading the Kiswa corpus Training dataset for processing
data = pd.read_csv("data/kiswa_test.csv")
# preview the top 5 rows of the dataset
data.head()

,date,id,news_heading_eng,news_heading_swa,news_type
0,2019-08-21,bbc8,the catholic church opposes the cancer vaccine...,kanisa katoliki linapinga chanjo ya saratani i...,Local News
1,2019-09-05,bbc25,free education is approved in drc for the firs...,elimu bila malipo imeidhinishwa nchini drc kwa...,Local News
2,2019-09-17,bbc30,a foreign entrepreneur was found guilty of def...,mjasiriamali wa kigeni alipatikana na hatia ya...,Local News
3,2019-10-04,bbc43,13 people died in a bus accident in kisumu kenya,watu 13 walifariki katika ajali ya basi huko k...,Local News
4,2019-10-12,bbc56,ineos challenge 1:59 : can you compete with el...,ineos challenge 1:59 : je unaweza kushindana n...,Sports news


In [17]:
# testing on this sample data
data.sample(n=5)

,date,id,news_heading_eng,news_heading_swa,news_type
229,2021-02-22,tl1593,relief for families corn price drop,unafuu kwa familia bei ya mahindi kushuka,Local News
51,2021-04-06,bbc483,why has jay z angered the muslims in lamu kenya ?,kwa nini jay z amewakasirisha waislamu huko la...,International news
90,2021-06-07,tl194,tanzania to begin vaccination exercise against...,tanzania kuanza zoezi la chanjo dhidi ya janga...,Health news
152,2021-09-11,tl839,the jungle foundation provides first aid train...,wakfu wa jungle watoa mafunzo ya huduma ya kwa...,Health news
199,2021-04-24,tl1292,the county takes actions to control dengue fever,kaunti yachukua hatua kudhibiti homa ya dengue,Health news


# Saving the classifier

In [4]:
# Fetching the current directory
import os
os.getcwd()

'/Users/sumitbajare/Documents/Sussex/Project'

In [5]:
# Chaging the directory 
os.chdir('/Users/sumitbajare/Documents/Sussex/Project/model')

In [ ]:
# Saving the classifier
kiswa_classifier.save('kiswa_corpus_classifier')

In [6]:
# Loading the classifier
import ktrain
predictor = ktrain.load_predictor('kiswa_corpus_classifier')

In [ ]:
res1=test('')

In [ ]:
# Predicting the news type 
predictor.predict(res1)